In [1]:
import numpy as np
import pandas as pd

from IPython.display import display, Markdown, Latex,  HTML
 

pd.options.display.float_format = '{:,.2f}'.format

display(HTML("<style>.dataframe td, .dataframe th { min-width:90px; }</style>"))

In [2]:
def proportional(C, b):
    display(Markdown('### Пропорційний податок'))
    display(Markdown('$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$'))
    proportional = b * C/b.sum()
    
    display(Markdown("---"))
    display(Markdown("$" +  " \qquad ".join([ f"x_{k+1} = {x:2.2f}" for k, x in enumerate(proportional)]) + "$"))
    
    return proportional

def rivnevy(C, b):
    
    display(Markdown('### Рівневий податок'))
    
    rivnevy = np.full_like(b, 0.0, dtype=np.double)
    
    markdown = []
    
    for i in range(len(b)):
        lamb = (1 / (len(b) - i)) * (b[i:].sum() - C)
        if lamb >= b[i]:
            
            markdown.append(
                '$$'
                '\lambda_{0} = \dfrac{{1}}{{n-{0}}} (\sum^{1}_{{i={2}}}b_i - C ) '
                '= \dfrac{{1}}{3} ({5} - {4}) '
                '= \dfrac{{{6}}}{{{3}}} \qquad '
                '\lambda_{0} (\dfrac{{{6}}}{{{3}}}) \geqslant b_{2} ({7}) \quad \Rightarrow x_{{{2}}} = 0'
                '$$'.format(
                    i, len(b), i+1, len(b)-i, C, b[i:].sum(), b[i:].sum() - C, b[i]
                )
            )
            
            rivnevy[i] = 0
            continue
        
        markdown.append(
                '$$'
                '\lambda_{0} = \dfrac{{1}}{{n-{0}}} (\sum^{1}_{{i={2}}}b_i - C ) '
                '= \dfrac{{1}}{3} ({5} - {4}) '
                '= \dfrac{{{6}}}{{{3}}} \qquad '
                '\lambda_{0}(\dfrac{{{6}}}{{{3}}}) < b_{2} ({7}) \quad \Rightarrow \lambda_{0} = \lambda^*'
                '\qquad x_i = b_i - \lambda^* \quad i=\overline{{{3},n}}'
                '$$'.format(
                    i, len(b), i+1, len(b)-i, C, b[i:].sum(), b[i:].sum() - C, b[i]
                )
        )
        
        rivnevy[i:] = b[i:] - lamb
        break
    
    
    display(Markdown( "\n".join(markdown) ))
    
    display(Markdown("---"))
    display(Markdown("$" +  " \qquad ".join([ f"x_{k+1} = {x:2.2f}" for k, x in enumerate(rivnevy)]) + "$"))
   
    return rivnevy

def podushny(C, b):
    
    display(Markdown('### Подушний податок'))
    
    podushny = np.full_like(b, 0.0, dtype=np.double)
    
    markdown = []
    
    for i in range(len(b)):
        lamb = (C - podushny.sum()) / (len(b) - i)
        
        
        if lamb >= b[i]:
            podushny[i] = b[i]
            
            markdown.append(
                '$$'
                '\lambda_{0} = \dfrac{{C - \sum_{{i=1}}^{0} x_i }}{{n-{0}}} \qquad '
                '\lambda_{0} (\dfrac{{{6}}}{{{3}}}) \geqslant b_{2} ({7}) \quad \Rightarrow x_{{{2}}} = b_{{{2}}} = {7}'
                '$$'.format(
                    i, len(b), i+1, len(b)-i, C, b[i:].sum(), b[i:].sum() - C, b[i]
                )
            )
                
            continue

        podushny[i:] = lamb
        
        markdown.append(
                '$$'
                '\lambda_{0} = \dfrac{{C - \sum_{{i=1}}^{0} x_i }}{{n-{0}}} \qquad '
                '\lambda_{0} (\dfrac{{{6}}}{{{3}}}) \leq b_{2} ({7}) \quad '
                '\Rightarrow \lambda_{0} = \lambda^* = x_{{i}} \quad i=\overline{{{2},n}}'
                '$$'.format(
                    i, len(b), i+1, len(b)-i, C, b[i:].sum(), b[i:].sum() - C, b[i]
                )
        )
            
        break
    
    display(Markdown( "\n".join(markdown) ))
    display(Markdown("---"))
    display(Markdown("$" +  " \qquad ".join([ f"x_{k+1} = {x:2.2f}" for k, x in enumerate(podushny)]) + "$"))
    
    return podushny

def nCore(C, b):
    core = np.full_like(b, 0.0, dtype=np.double)

    """ default case C > b.sum()/2"""
    
    display(Markdown('### N-Ядро'))
    
    markdown = []
    
    def nCoreLess(C, b, core):
        
        markdown.append(
                '$$'
                'C < \dfrac{{\sum b }}{{2}}'
                '$$'.format(
                    C, b.sum()/2
                )
        )
        
        for i in range(len(b)):
            lamb = (C - core.sum()) / (len(b) - i)
           
                
            if lamb > b[i]/2:
                markdown.append(
                    '$$'
                    '\lambda_{0} = \dfrac{{C-\sum_{{i=1}}^{0} x_i}}{{n-{0}}}'
                    ' = \dfrac{{{2}}}{{{1}}} \geqslant \dfrac{{b_{3}}}{{2}} ({4}) \qquad '
                    ' x_{3} = \dfrac{{b_{{{3}}}}}{{2}} = {4}'
                    '$$'.format(
                        i, len(b)-i, (C-core.sum()), i+1, b[i]/2
                    )
                )
                    
                core[i] = b[i]/2
                continue
            
            markdown.append(
                '$$'
                '\lambda_{0} = \dfrac{{C-\sum_{{i=1}}^{0} x_i}}{{n-{0}}}'
                ' = \dfrac{{{2}}}{{{1}}} < \dfrac{{b_{3}}}{{2}} ({4}) \\quad'
                '\lambda_{0} = \lambda^* = x_i \quad i=\overline{{{3},n}}'
                '$$'.format(
                    i, len(b)-i, (C-core.sum()), i+1, b[i]/2
                )
            )
            
            core[i:] = lamb
            break
        return core

    def nCoreMore(C, b, core):
        markdown.append(
                '$$'
                'C \geqslant \dfrac{{\sum b }}{{2}}'
                '$$'.format(
                    C, b.sum()/2
                )
        )
            
        for i in range(len(b)):
            lamb = (1/(len(b)-i)) * (b.sum() - C - core.sum())
            
            if lamb > b[i]/2:
                
                markdown.append(
                    '$$'
                    '\lambda_{0} = \dfrac{{1}}{{n-{0}}} * ( \sum^{5}_{{i=1}} b_i - (C - \sum^{0}_{{i=1}} x_i) ) = '
                    '\dfrac{{({6} - ({7}-{8}))}}{{{1}}}  = '
                    '\dfrac{{{6}-{2}}}{{{1}}} = '
                     '\dfrac{{{9}}}{{{1}}} '
                    ' \geqslant \dfrac{{b_{3}}}{{2}} ({4}) \qquad '
                    ' x_{3} = \dfrac{{b_{{{3}}}}}{{2}} = {4}'
                    '$$'.format(
                        i, len(b)-i, (C-core.sum()), i+1, b[i]/2, len(b), b.sum(), C, core.sum(), b.sum()-(C-core.sum())
                    )
                )

                
                core[i] = b[i]/2
                continue
            
            markdown.append(
                '$$'
                '\lambda_{0} = \dfrac{{1}}{{n-{0}}} * ( \sum^{5}_{{i=1}} b_i - (C - \sum^{0}_{{i=1}} x_i) ) = '
                '\dfrac{{({6} - ({7}-{8}))}}{{{1}}}  = '
                '\dfrac{{{6}-{2}}}{{{1}}} = '
                 '\dfrac{{{9}}}{{{1}}} '
                ' < \dfrac{{b_{3}}}{{2}} ({4}) \qquad '
                ' \lambda_{0} = \lambda^* \qquad x_{{i}} = b_{{i}}-\lambda^* \quad i=\overline{{{3},n}}'
                '$$'.format(
                    i, len(b)-i, (C-core.sum()), i+1, b[i]/2, len(b), b.sum(), C, core.sum(), b.sum()-(C-core.sum())
                )
            )
                
            core[i:] = b[i:] - lamb
            break

        return core

    result = (nCoreLess if C < b.sum()/2 else nCoreMore)(C, b, core)
    
    display(Markdown( "\n".join(markdown) ))
    display(Markdown("---"))
    display(Markdown("$" +  " \qquad ".join([ f"x_{k+1} = {x:2.2f}" for k, x in enumerate(result)]) + "$"))
    
    return result

def report(C, b):
    df = pd.DataFrame(data=b, 
                      columns = ["Dohody"],
                      index=[ f"$b_{x} ({b[x-1]}) / x_{x} $" for x in range(1,len(b)+1)]
    )
    df['Подушний']       = podushny(C, b)
    df['Рівневий']       = rivnevy(C, b)
    df['n-Core']        = nCore(C, b)
    df['Пропопрційний']  = proportional(C, b)
    
    df = df.T
    df.drop(['Dohody'], inplace=True)
    df['$\sum x_i $'] = df.sum(axis=1)
    
    
    display(df)
    
def type1(N, b, C):
    display(Markdown(f"### Задача {N}"))
    
    mdstr = ", ".join([ f"$x_{x+1}$ = {b[x]}" for x in range(len(b))])
    display(Markdown( f"Нехай є n={len(b)} учасників спільної діяльності."
                      f"Припустимо що доходи учасників дорівнюють відповідно {mdstr}, а загальні витрати складають С={C}"))
    
    report(C, b)
    
def type2(N, b, C):
    display(Markdown(f"### Задача {N}"))
    
    mdstr = ", ".join([ f"{b[x]}" for x in range(len(b))])
    display(Markdown( f"Нехай є фірма, яка збанкротувала. Її залишкове майно складає {C} грн., а борги "
                      f"кредиторам складають відповідно {mdstr} грн. "
                      "Знайти справедливе розподілення майна фірми між кредиторами."))
    
    nCore(C, b)

## Контрольна робота 8

In [3]:
type1("1.1 (a)", np.array([ 4, 8, 12, 20, 26 ]), 30)

### Задача 1.1 (a)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 4, $x_2$ = 8, $x_3$ = 12, $x_4$ = 20, $x_5$ = 26, а загальні витрати складають С=30

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{40}{5}) \geqslant b_1 (4) \quad \Rightarrow x_{1} = b_{1} = 4$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{36}{4}) \leq b_2 (8) \quad \Rightarrow \lambda_1 = \lambda^* = x_{i} \quad i=\overline{2,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 1.71 \qquad x_2 = 3.43 \qquad x_3 = 5.14 \qquad x_4 = 8.57 \qquad x_5 = 11.14$

,$b_1 (4) / x_1 $,$b_2 (8) / x_2 $,$b_3 (12) / x_3 $,$b_4 (20) / x_4 $,$b_5 (26) / x_5 $,$\sum x_i $
Подушний,4.00,6.50,6.50,6.50,6.50,30.00
Рівневий,0.00,0.00,2.67,10.67,16.67,30.00
n-Core,2.00,4.00,6.00,9.00,9.00,30.00
Пропопрційний,1.71,3.43,5.14,8.57,11.14,30.00


In [4]:
type1("1.1 (б)", np.array([ 4, 8, 12, 20, 26 ]), 50)

### Задача 1.1 (б)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 4, $x_2$ = 8, $x_3$ = 12, $x_4$ = 20, $x_5$ = 26, а загальні витрати складають С=50

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{20}{5}) \geqslant b_1 (4) \quad \Rightarrow x_{1} = b_{1} = 4$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{16}{4}) \geqslant b_2 (8) \quad \Rightarrow x_{2} = b_{2} = 8$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{8}{3}) \geqslant b_3 (12) \quad \Rightarrow x_{3} = b_{3} = 12$$
$$\lambda_3 = \dfrac{C - \sum_{i=1}^3 x_i }{n-3} \qquad \lambda_3 (\dfrac{-4}{2}) \leq b_4 (20) \quad \Rightarrow \lambda_3 = \lambda^* = x_{i} \quad i=\overline{4,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 2.86 \qquad x_2 = 5.71 \qquad x_3 = 8.57 \qquad x_4 = 14.29 \qquad x_5 = 18.57$

,$b_1 (4) / x_1 $,$b_2 (8) / x_2 $,$b_3 (12) / x_3 $,$b_4 (20) / x_4 $,$b_5 (26) / x_5 $,$\sum x_i $
Подушний,4.00,8.00,12.00,13.00,13.00,50.00
Рівневий,0.00,4.00,8.00,16.00,22.00,50.00
n-Core,2.00,4.00,7.33,15.33,21.33,50.00
Пропопрційний,2.86,5.71,8.57,14.29,18.57,50.00


In [5]:
type2("1.2", np.array([ 6000, 8000, 10000 ]), 15000)

### Задача 1.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 15000 грн., а борги кредиторам складають відповідно 6000, 8000, 10000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C \geqslant \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{1}{n-0} * ( \sum^3_{i=1} b_i - (C - \sum^0_{i=1} x_i) ) = \dfrac{(24000 - (15000-0.0))}{3}  = \dfrac{24000-15000.0}{3} = \dfrac{9000.0}{3}  < \dfrac{b_1}{2} (3000.0) \qquad  \lambda_0 = \lambda^* \qquad x_{i} = b_{i}-\lambda^* \quad i=\overline{1,n}$$

---

$x_1 = 3000.00 \qquad x_2 = 5000.00 \qquad x_3 = 7000.00$

In [6]:
type1("2.2 (a)", np.array([ 10, 12, 18, 20, 30 ]), 45)

### Задача 2.2 (a)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 10, $x_2$ = 12, $x_3$ = 18, $x_4$ = 20, $x_5$ = 30, а загальні витрати складають С=45

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{45}{5}) \leq b_1 (10) \quad \Rightarrow \lambda_0 = \lambda^* = x_{i} \quad i=\overline{1,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 5.00 \qquad x_2 = 6.00 \qquad x_3 = 9.00 \qquad x_4 = 10.00 \qquad x_5 = 15.00$

,$b_1 (10) / x_1 $,$b_2 (12) / x_2 $,$b_3 (18) / x_3 $,$b_4 (20) / x_4 $,$b_5 (30) / x_5 $,$\sum x_i $
Подушний,9.00,9.00,9.00,9.00,9.00,45.00
Рівневий,1.00,3.00,9.00,11.00,21.00,45.00
n-Core,5.00,6.00,9.00,10.00,15.00,45.00
Пропопрційний,5.00,6.00,9.00,10.00,15.00,45.00


In [7]:
type1("2.2 (б)", np.array([ 10, 12, 18, 20, 30 ]), 75)

### Задача 2.2 (б)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 10, $x_2$ = 12, $x_3$ = 18, $x_4$ = 20, $x_5$ = 30, а загальні витрати складають С=75

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{15}{5}) \geqslant b_1 (10) \quad \Rightarrow x_{1} = b_{1} = 10$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{5}{4}) \geqslant b_2 (12) \quad \Rightarrow x_{2} = b_{2} = 12$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{-7}{3}) \leq b_3 (18) \quad \Rightarrow \lambda_2 = \lambda^* = x_{i} \quad i=\overline{3,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 8.33 \qquad x_2 = 10.00 \qquad x_3 = 15.00 \qquad x_4 = 16.67 \qquad x_5 = 25.00$

,$b_1 (10) / x_1 $,$b_2 (12) / x_2 $,$b_3 (18) / x_3 $,$b_4 (20) / x_4 $,$b_5 (30) / x_5 $,$\sum x_i $
Подушний,10.00,12.00,17.67,17.67,17.67,75.00
Рівневий,7.00,9.00,15.00,17.00,27.00,75.00
n-Core,7.00,9.00,15.00,17.00,27.00,75.00
Пропопрційний,8.33,10.00,15.00,16.67,25.00,75.00


In [8]:
type2("2.2", np.array([ 4000, 6000, 8000 ]), 10000)

### Задача 2.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 10000 грн., а борги кредиторам складають відповідно 4000, 6000, 8000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C \geqslant \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{1}{n-0} * ( \sum^3_{i=1} b_i - (C - \sum^0_{i=1} x_i) ) = \dfrac{(18000 - (10000-0.0))}{3}  = \dfrac{18000-10000.0}{3} = \dfrac{8000.0}{3}  \geqslant \dfrac{b_1}{2} (2000.0) \qquad  x_1 = \dfrac{b_{1}}{2} = 2000.0$$
$$\lambda_1 = \dfrac{1}{n-1} * ( \sum^3_{i=1} b_i - (C - \sum^1_{i=1} x_i) ) = \dfrac{(18000 - (10000-2000.0))}{2}  = \dfrac{18000-8000.0}{2} = \dfrac{10000.0}{2}  < \dfrac{b_2}{2} (3000.0) \qquad  \lambda_1 = \lambda^* \qquad x_{i} = b_{i}-\lambda^* \quad i=\overline{2,n}$$

---

$x_1 = 2000.00 \qquad x_2 = 3000.00 \qquad x_3 = 5000.00$

In [9]:
type1("3.1 (a)", np.array([ 6, 10, 14, 20, 24 ]), 30)

### Задача 3.1 (a)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 6, $x_2$ = 10, $x_3$ = 14, $x_4$ = 20, $x_5$ = 24, а загальні витрати складають С=30

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{44}{5}) \geqslant b_1 (6) \quad \Rightarrow x_{1} = b_{1} = 6$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{38}{4}) \leq b_2 (10) \quad \Rightarrow \lambda_1 = \lambda^* = x_{i} \quad i=\overline{2,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 2.43 \qquad x_2 = 4.05 \qquad x_3 = 5.68 \qquad x_4 = 8.11 \qquad x_5 = 9.73$

,$b_1 (6) / x_1 $,$b_2 (10) / x_2 $,$b_3 (14) / x_3 $,$b_4 (20) / x_4 $,$b_5 (24) / x_5 $,$\sum x_i $
Подушний,6.00,6.00,6.00,6.00,6.00,30.00
Рівневий,0.00,0.50,4.50,10.50,14.50,30.00
n-Core,3.00,5.00,7.00,7.50,7.50,30.00
Пропопрційний,2.43,4.05,5.68,8.11,9.73,30.00


In [10]:
type1("3.2 (б)", np.array([ 6, 10, 14, 20, 24 ]), 60)

### Задача 3.2 (б)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 6, $x_2$ = 10, $x_3$ = 14, $x_4$ = 20, $x_5$ = 24, а загальні витрати складають С=60

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{14}{5}) \geqslant b_1 (6) \quad \Rightarrow x_{1} = b_{1} = 6$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{8}{4}) \geqslant b_2 (10) \quad \Rightarrow x_{2} = b_{2} = 10$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{-2}{3}) \geqslant b_3 (14) \quad \Rightarrow x_{3} = b_{3} = 14$$
$$\lambda_3 = \dfrac{C - \sum_{i=1}^3 x_i }{n-3} \qquad \lambda_3 (\dfrac{-16}{2}) \leq b_4 (20) \quad \Rightarrow \lambda_3 = \lambda^* = x_{i} \quad i=\overline{4,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 4.86 \qquad x_2 = 8.11 \qquad x_3 = 11.35 \qquad x_4 = 16.22 \qquad x_5 = 19.46$

,$b_1 (6) / x_1 $,$b_2 (10) / x_2 $,$b_3 (14) / x_3 $,$b_4 (20) / x_4 $,$b_5 (24) / x_5 $,$\sum x_i $
Подушний,6.00,10.00,14.00,15.00,15.00,60.00
Рівневий,3.20,7.20,11.20,17.20,21.20,60.00
n-Core,3.20,7.20,11.20,17.20,21.20,60.00
Пропопрційний,4.86,8.11,11.35,16.22,19.46,60.00


In [11]:
type2("3.2", np.array([ 6000, 8000, 12000 ]), 12000)

### Задача 3.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 12000 грн., а борги кредиторам складають відповідно 6000, 8000, 12000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C < \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{C-\sum_{i=1}^0 x_i}{n-0} = \dfrac{12000.0}{3} \geqslant \dfrac{b_1}{2} (3000.0) \qquad  x_1 = \dfrac{b_{1}}{2} = 3000.0$$
$$\lambda_1 = \dfrac{C-\sum_{i=1}^1 x_i}{n-1} = \dfrac{9000.0}{2} \geqslant \dfrac{b_2}{2} (4000.0) \qquad  x_2 = \dfrac{b_{2}}{2} = 4000.0$$
$$\lambda_2 = \dfrac{C-\sum_{i=1}^2 x_i}{n-2} = \dfrac{5000.0}{1} < \dfrac{b_3}{2} (6000.0) \quad\lambda_2 = \lambda^* = x_i \quad i=\overline{3,n}$$

---

$x_1 = 3000.00 \qquad x_2 = 4000.00 \qquad x_3 = 5000.00$

In [12]:
type1("4.1 (a)", np.array([ 8, 12, 20, 24, 32 ]), 36)

### Задача 4.1 (a)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 8, $x_2$ = 12, $x_3$ = 20, $x_4$ = 24, $x_5$ = 32, а загальні витрати складають С=36

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{60}{5}) \leq b_1 (8) \quad \Rightarrow \lambda_0 = \lambda^* = x_{i} \quad i=\overline{1,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 3.00 \qquad x_2 = 4.50 \qquad x_3 = 7.50 \qquad x_4 = 9.00 \qquad x_5 = 12.00$

,$b_1 (8) / x_1 $,$b_2 (12) / x_2 $,$b_3 (20) / x_3 $,$b_4 (24) / x_4 $,$b_5 (32) / x_5 $,$\sum x_i $
Подушний,7.20,7.20,7.20,7.20,7.20,36.00
Рівневий,0.00,0.00,6.67,10.67,18.67,36.00
n-Core,4.00,6.00,8.67,8.67,8.67,36.00
Пропопрційний,3.00,4.50,7.50,9.00,12.00,36.00


In [13]:
type1("4.1 (б)", np.array([ 8, 12, 20, 24, 32 ]), 70)

### Задача 4.1 (б)

Нехай є n=5 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 8, $x_2$ = 12, $x_3$ = 20, $x_4$ = 24, $x_5$ = 32, а загальні витрати складають С=70

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{26}{5}) \geqslant b_1 (8) \quad \Rightarrow x_{1} = b_{1} = 8$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{18}{4}) \geqslant b_2 (12) \quad \Rightarrow x_{2} = b_{2} = 12$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{6}{3}) \leq b_3 (20) \quad \Rightarrow \lambda_2 = \lambda^* = x_{i} \quad i=\overline{3,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 5.83 \qquad x_2 = 8.75 \qquad x_3 = 14.58 \qquad x_4 = 17.50 \qquad x_5 = 23.33$

,$b_1 (8) / x_1 $,$b_2 (12) / x_2 $,$b_3 (20) / x_3 $,$b_4 (24) / x_4 $,$b_5 (32) / x_5 $,$\sum x_i $
Подушний,8.00,12.00,16.67,16.67,16.67,70.00
Рівневий,2.80,6.80,14.80,18.80,26.80,70.00
n-Core,4.00,6.50,14.50,18.50,26.50,70.00
Пропопрційний,5.83,8.75,14.58,17.50,23.33,70.00


In [14]:
type2("4.2", np.array([ 4000, 6000, 10000 ]), 16000)

### Задача 4.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 16000 грн., а борги кредиторам складають відповідно 4000, 6000, 10000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C \geqslant \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{1}{n-0} * ( \sum^3_{i=1} b_i - (C - \sum^0_{i=1} x_i) ) = \dfrac{(20000 - (16000-0.0))}{3}  = \dfrac{20000-16000.0}{3} = \dfrac{4000.0}{3}  < \dfrac{b_1}{2} (2000.0) \qquad  \lambda_0 = \lambda^* \qquad x_{i} = b_{i}-\lambda^* \quad i=\overline{1,n}$$

---

$x_1 = 2666.67 \qquad x_2 = 4666.67 \qquad x_3 = 8666.67$

In [15]:
type1("5.1 (a)", np.array([ 10, 12, 18, 20, 30, 36 ]), 50)

### Задача 5.1 (a)

Нехай є n=6 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 10, $x_2$ = 12, $x_3$ = 18, $x_4$ = 20, $x_5$ = 30, $x_6$ = 36, а загальні витрати складають С=50

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{76}{6}) \leq b_1 (10) \quad \Rightarrow \lambda_0 = \lambda^* = x_{i} \quad i=\overline{1,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 3.97 \qquad x_2 = 4.76 \qquad x_3 = 7.14 \qquad x_4 = 7.94 \qquad x_5 = 11.90 \qquad x_6 = 14.29$

,$b_1 (10) / x_1 $,$b_2 (12) / x_2 $,$b_3 (18) / x_3 $,$b_4 (20) / x_4 $,$b_5 (30) / x_5 $,$b_6 (36) / x_6 $,$\sum x_i $
Подушний,8.33,8.33,8.33,8.33,8.33,8.33,50.00
Рівневий,0.00,0.00,4.50,6.50,16.50,22.50,50.00
n-Core,5.00,6.00,9.00,10.00,10.00,10.00,50.00
Пропопрційний,3.97,4.76,7.14,7.94,11.90,14.29,50.00


In [16]:
type1("5.1 (б)", np.array([ 10, 12, 18, 20, 30, 36 ]), 90)

### Задача 5.1 (б)

Нехай є n=6 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 10, $x_2$ = 12, $x_3$ = 18, $x_4$ = 20, $x_5$ = 30, $x_6$ = 36, а загальні витрати складають С=90

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{36}{6}) \geqslant b_1 (10) \quad \Rightarrow x_{1} = b_{1} = 10$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{26}{5}) \geqslant b_2 (12) \quad \Rightarrow x_{2} = b_{2} = 12$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{14}{4}) \leq b_3 (18) \quad \Rightarrow \lambda_2 = \lambda^* = x_{i} \quad i=\overline{3,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 7.14 \qquad x_2 = 8.57 \qquad x_3 = 12.86 \qquad x_4 = 14.29 \qquad x_5 = 21.43 \qquad x_6 = 25.71$

,$b_1 (10) / x_1 $,$b_2 (12) / x_2 $,$b_3 (18) / x_3 $,$b_4 (20) / x_4 $,$b_5 (30) / x_5 $,$b_6 (36) / x_6 $,$\sum x_i $
Подушний,10.00,12.00,17.00,17.00,17.00,17.00,90.00
Рівневий,4.00,6.00,12.00,14.00,24.00,30.00,90.00
n-Core,5.00,6.00,11.75,13.75,23.75,29.75,90.00
Пропопрційний,7.14,8.57,12.86,14.29,21.43,25.71,90.00


In [17]:
type2("5.2", np.array([ 4000, 6000, 18000 ]), 20000)

### Задача 5.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 20000 грн., а борги кредиторам складають відповідно 4000, 6000, 18000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C \geqslant \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{1}{n-0} * ( \sum^3_{i=1} b_i - (C - \sum^0_{i=1} x_i) ) = \dfrac{(28000 - (20000-0.0))}{3}  = \dfrac{28000-20000.0}{3} = \dfrac{8000.0}{3}  \geqslant \dfrac{b_1}{2} (2000.0) \qquad  x_1 = \dfrac{b_{1}}{2} = 2000.0$$
$$\lambda_1 = \dfrac{1}{n-1} * ( \sum^3_{i=1} b_i - (C - \sum^1_{i=1} x_i) ) = \dfrac{(28000 - (20000-2000.0))}{2}  = \dfrac{28000-18000.0}{2} = \dfrac{10000.0}{2}  < \dfrac{b_2}{2} (3000.0) \qquad  \lambda_1 = \lambda^* \qquad x_{i} = b_{i}-\lambda^* \quad i=\overline{2,n}$$

---

$x_1 = 2000.00 \qquad x_2 = 3000.00 \qquad x_3 = 15000.00$

In [18]:
type1("6.1 (a)", np.array([ 4, 8, 12, 20, 26, 30 ]), 48)

### Задача 6.1 (a)

Нехай є n=6 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 4, $x_2$ = 8, $x_3$ = 12, $x_4$ = 20, $x_5$ = 26, $x_6$ = 30, а загальні витрати складають С=48

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{52}{6}) \geqslant b_1 (4) \quad \Rightarrow x_{1} = b_{1} = 4$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{48}{5}) \geqslant b_2 (8) \quad \Rightarrow x_{2} = b_{2} = 8$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{40}{4}) \leq b_3 (12) \quad \Rightarrow \lambda_2 = \lambda^* = x_{i} \quad i=\overline{3,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 1.92 \qquad x_2 = 3.84 \qquad x_3 = 5.76 \qquad x_4 = 9.60 \qquad x_5 = 12.48 \qquad x_6 = 14.40$

,$b_1 (4) / x_1 $,$b_2 (8) / x_2 $,$b_3 (12) / x_3 $,$b_4 (20) / x_4 $,$b_5 (26) / x_5 $,$b_6 (30) / x_6 $,$\sum x_i $
Подушний,4.00,8.00,9.00,9.00,9.00,9.00,48.00
Рівневий,0.00,0.00,2.00,10.00,16.00,20.00,48.00
n-Core,2.00,4.00,6.00,10.00,13.00,13.00,48.00
Пропопрційний,1.92,3.84,5.76,9.60,12.48,14.40,48.00


In [19]:
type1("6.1 (б)", np.array([ 4, 8, 12, 20, 26, 30 ]), 84)

### Задача 6.1 (б)

Нехай є n=6 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 4, $x_2$ = 8, $x_3$ = 12, $x_4$ = 20, $x_5$ = 26, $x_6$ = 30, а загальні витрати складають С=84

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{16}{6}) \geqslant b_1 (4) \quad \Rightarrow x_{1} = b_{1} = 4$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{12}{5}) \geqslant b_2 (8) \quad \Rightarrow x_{2} = b_{2} = 8$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{4}{4}) \geqslant b_3 (12) \quad \Rightarrow x_{3} = b_{3} = 12$$
$$\lambda_3 = \dfrac{C - \sum_{i=1}^3 x_i }{n-3} \qquad \lambda_3 (\dfrac{-8}{3}) \geqslant b_4 (20) \quad \Rightarrow x_{4} = b_{4} = 20$$
$$\lambda_4 = \dfrac{C - \sum_{i=1}^4 x_i }{n-4} \qquad \lambda_4 (\dfrac{-28}{2}) \leq b_5 (26) \quad \Rightarrow \lambda_4 = \lambda^* = x_{i} \quad i=\overline{5,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 3.36 \qquad x_2 = 6.72 \qquad x_3 = 10.08 \qquad x_4 = 16.80 \qquad x_5 = 21.84 \qquad x_6 = 25.20$

,$b_1 (4) / x_1 $,$b_2 (8) / x_2 $,$b_3 (12) / x_3 $,$b_4 (20) / x_4 $,$b_5 (26) / x_5 $,$b_6 (30) / x_6 $,$\sum x_i $
Подушний,4.00,8.00,12.00,20.00,20.00,20.00,84.00
Рівневий,1.33,5.33,9.33,17.33,23.33,27.33,84.00
n-Core,2.00,5.20,9.20,17.20,23.20,27.20,84.00
Пропопрційний,3.36,6.72,10.08,16.80,21.84,25.20,84.00


In [20]:
type2("6.2", np.array([ 6000, 8000, 10000 ]), 15000)

### Задача 6.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 15000 грн., а борги кредиторам складають відповідно 6000, 8000, 10000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C \geqslant \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{1}{n-0} * ( \sum^3_{i=1} b_i - (C - \sum^0_{i=1} x_i) ) = \dfrac{(24000 - (15000-0.0))}{3}  = \dfrac{24000-15000.0}{3} = \dfrac{9000.0}{3}  < \dfrac{b_1}{2} (3000.0) \qquad  \lambda_0 = \lambda^* \qquad x_{i} = b_{i}-\lambda^* \quad i=\overline{1,n}$$

---

$x_1 = 3000.00 \qquad x_2 = 5000.00 \qquad x_3 = 7000.00$

In [21]:
type1("7.1 (a)", np.array([ 6, 10, 14, 20, 24, 36 ]), 50)

### Задача 7.1 (a)

Нехай є n=6 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 6, $x_2$ = 10, $x_3$ = 14, $x_4$ = 20, $x_5$ = 24, $x_6$ = 36, а загальні витрати складають С=50

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{60}{6}) \geqslant b_1 (6) \quad \Rightarrow x_{1} = b_{1} = 6$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{54}{5}) \leq b_2 (10) \quad \Rightarrow \lambda_1 = \lambda^* = x_{i} \quad i=\overline{2,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 2.73 \qquad x_2 = 4.55 \qquad x_3 = 6.36 \qquad x_4 = 9.09 \qquad x_5 = 10.91 \qquad x_6 = 16.36$

,$b_1 (6) / x_1 $,$b_2 (10) / x_2 $,$b_3 (14) / x_3 $,$b_4 (20) / x_4 $,$b_5 (24) / x_5 $,$b_6 (36) / x_6 $,$\sum x_i $
Подушний,6.00,8.80,8.80,8.80,8.80,8.80,50.00
Рівневий,0.00,0.00,3.00,9.00,13.00,25.00,50.00
n-Core,3.00,5.00,7.00,10.00,12.00,13.00,50.00
Пропопрційний,2.73,4.55,6.36,9.09,10.91,16.36,50.00


In [22]:
type1("7.1 (б)", np.array([ 6, 10, 14, 20, 24, 36 ]), 80)

### Задача 7.1 (б)

Нехай є n=6 учасників спільної діяльності.Припустимо що доходи учасників дорівнюють відповідно $x_1$ = 6, $x_2$ = 10, $x_3$ = 14, $x_4$ = 20, $x_5$ = 24, $x_6$ = 36, а загальні витрати складають С=80

### Подушний податок

$$\lambda_0 = \dfrac{C - \sum_{i=1}^0 x_i }{n-0} \qquad \lambda_0 (\dfrac{30}{6}) \geqslant b_1 (6) \quad \Rightarrow x_{1} = b_{1} = 6$$
$$\lambda_1 = \dfrac{C - \sum_{i=1}^1 x_i }{n-1} \qquad \lambda_1 (\dfrac{24}{5}) \geqslant b_2 (10) \quad \Rightarrow x_{2} = b_{2} = 10$$
$$\lambda_2 = \dfrac{C - \sum_{i=1}^2 x_i }{n-2} \qquad \lambda_2 (\dfrac{14}{4}) \geqslant b_3 (14) \quad \Rightarrow x_{3} = b_{3} = 14$$
$$\lambda_3 = \dfrac{C - \sum_{i=1}^3 x_i }{n-3} \qquad \lambda_3 (\dfrac{0}{3}) \leq b_4 (20) \quad \Rightarrow \lambda_3 = \lambda^* = x_{i} \quad i=\overline{4,n}$$

---

...

### Пропорційний податок

$$ x_i = b_i \dfrac{C}{\sum_{i=1}^{n}} b_i \qquad i=\overline{1,n}$$

---

$x_1 = 4.36 \qquad x_2 = 7.27 \qquad x_3 = 10.18 \qquad x_4 = 14.55 \qquad x_5 = 17.45 \qquad x_6 = 26.18$

,$b_1 (6) / x_1 $,$b_2 (10) / x_2 $,$b_3 (14) / x_3 $,$b_4 (20) / x_4 $,$b_5 (24) / x_5 $,$b_6 (36) / x_6 $,$\sum x_i $
Подушний,6.00,10.00,14.00,16.67,16.67,16.67,80.00
Рівневий,1.00,5.00,9.00,15.00,19.00,31.00,80.00
n-Core,3.00,5.00,8.50,14.50,18.50,30.50,80.00
Пропопрційний,4.36,7.27,10.18,14.55,17.45,26.18,80.00


In [23]:
type2("7.2", np.array([ 6000, 8000, 12000 ]), 12000)

### Задача 7.2

Нехай є фірма, яка збанкротувала. Її залишкове майно складає 12000 грн., а борги кредиторам складають відповідно 6000, 8000, 12000 грн. Знайти справедливе розподілення майна фірми між кредиторами.

### N-Ядро

$$C < \dfrac{\sum b }{2}$$
$$\lambda_0 = \dfrac{C-\sum_{i=1}^0 x_i}{n-0} = \dfrac{12000.0}{3} \geqslant \dfrac{b_1}{2} (3000.0) \qquad  x_1 = \dfrac{b_{1}}{2} = 3000.0$$
$$\lambda_1 = \dfrac{C-\sum_{i=1}^1 x_i}{n-1} = \dfrac{9000.0}{2} \geqslant \dfrac{b_2}{2} (4000.0) \qquad  x_2 = \dfrac{b_{2}}{2} = 4000.0$$
$$\lambda_2 = \dfrac{C-\sum_{i=1}^2 x_i}{n-2} = \dfrac{5000.0}{1} < \dfrac{b_3}{2} (6000.0) \quad\lambda_2 = \lambda^* = x_i \quad i=\overline{3,n}$$

---

$x_1 = 3000.00 \qquad x_2 = 4000.00 \qquad x_3 = 5000.00$